# Import package


In [1]:
import requests
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from autograd import grad, jacobian
from scipy.optimize import line_search, minimize
import datetime
from typing import List
from numba import njit

# Data import and processing


In [2]:
settledate = pd.to_datetime(datetime.date.today())
bonds = pd.DataFrame(
    json.loads(
        requests.get(
            "https://asx.api.markitdigital.com/asx-research/1.0/bonds/government/exchange-traded?height=179&width=690"
        ).content
    )["data"]["items"]
)
bonds["maturity"] = bonds["securityDescription"].str.extract(r"(\d{2}-\d{2}-\d{2})")[0]
bonds["maturity"] = pd.to_datetime(bonds["maturity"], format="%d-%m-%y")
bonds = bonds[["maturity", "couponPercent", "priceBid", "priceAsk"]]
bonds["couponPercent"] = bonds["couponPercent"].astype(float) / 100
bonds.dropna(inplace=True)
bonds.sort_values("maturity", inplace=True)
bonds.reset_index(drop=True, inplace=True)
bonds

,maturity,couponPercent,priceBid,priceAsk
0,2025-11-21,0.0025,98.21,99.00
1,2026-04-21,0.0425,100.80,103.00
2,2027-04-21,0.0475,102.82,103.32
3,2027-11-21,0.0275,97.00,100.06
4,2028-05-21,0.0225,97.80,98.06
5,2029-04-21,0.0325,97.00,99.82
6,2033-04-21,0.0450,102.90,105.00
7,2033-11-21,0.0300,93.50,95.00
8,2037-04-21,0.0375,94.40,95.80
9,2039-06-21,0.0325,86.50,92.40


# Support function


In [3]:
def year_transform(date: pd.Timestamp) -> float:
    """Transform date to actual year"""
    if date == settledate:
        return 0
    years = date.year - settledate.year
    check_date = pd.Timestamp(
        year=date.year,
        month=settledate.month,
        day=settledate.day,
    )
    years += (date - check_date).days / abs(
        (
            check_date
            + pd.DateOffset(years=np.sign((date - check_date).days))
            - check_date
        ).days
    )
    return years


def coupon_date_generate(maturity: pd.Timestamp) -> np.ndarray:
    """Generate coupon dates"""
    coupon_dates = [maturity]
    while maturity - pd.DateOffset(months=6) > settledate:
        maturity -= pd.DateOffset(months=6)
        coupon_dates.append(maturity)
    return np.array(list(map(year_transform, coupon_dates[::-1])))

# Discount factor formula


In [4]:
@njit(cache=True)
def discount_factor(params: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Calculate discount factor"""
    f0, f1, f2, gamma = params
    return np.exp(
        -(
            f0 * t
            + f1 * (gamma - np.exp(-t / gamma) * gamma)
            + f2 * (gamma - np.exp(-t / gamma) * (t + gamma))
        )
    )

# Discount factor curve


In [5]:
def plot(params: np.ndarray):
    t = np.linspace(0, 30, 100000)
    dfs = discount_factor(params, t)
    plt.figure(figsize=(15, 6))
    plt.plot(t, dfs)
    plt.title("Discount Factor Curve")
    plt.xlabel("Years")
    plt.ylabel("Discount Factor")
    plt.grid()
    plt.show()

# Bond pricing formula


In [6]:
@njit(cache=True)
def bond_valuation(params: np.ndarray, t: np.ndarray, coupon: float) -> float:
    """Calculate bond valuation"""
    cf = np.ones_like(t) * coupon / 2
    cf[-1] += 1
    return 100 * np.sum(cf * discount_factor(params, t))

# Loss function


In [13]:
@njit(cache=True)
def loss_function(
    params: np.ndarray, t: np.ndarray, coupon: float, bid: float, ask: float
) -> float:
    """Objective function for optimization"""
    bond_price = bond_valuation(params, t, coupon)
    return (max(0, bond_price - ask) / ask) ** 2 + (max(0, bid - bond_price) / bid) ** 2

# Gradient function


In [ ]:
@njit(cache=True)
def discount_factor_gradient(
    params: np.ndarray, t: np.ndarray, element: int
) -> np.ndarray:
    """Calculate gradient of discount factor"""
    f0, f1, f2, gamma = params
    if element == 0:
        return -t * discount_factor(params, t)
    elif element == 1:
        return (-gamma + np.exp(-t / gamma) * gamma) * discount_factor(params, t)
    elif element == 2:
        return (-gamma + np.exp(-t / gamma) * (t + gamma)) * discount_factor(params, t)
    else:
        return (
            -f1 * (1 - np.exp(-t / gamma) - t * np.exp(-t / gamma) / gamma)
            - f2
            * (
                1
                - np.exp(-t / gamma)
                - t * np.exp(-t / gamma) * (t + gamma) / (gamma**2)
            )
        ) * discount_factor(params, t)


@njit(cache=True)
def bond_valuation_gradient(
    params: np.ndarray, t: np.ndarray, coupon: float, element: int
) -> float:
    """Calculate gradient of bond valuation"""
    cf = np.ones_like(t) * coupon / 2
    cf[-1] += 1
    return 100 * np.sum(cf * discount_factor_gradient(params, t, element))


@njit(cache=True)
def loss_function_component_gradient(
    params: np.ndarray,
    t: np.ndarray,
    coupon: float,
    bid: float,
    ask: float,
    element: int,
) -> float:
    """Calculate gradient of bond valuation"""
    vj = bond_valuation(params, t, coupon)
    if vj > ask:
        return (2 * (vj - ask) / (ask**2)) * bond_valuation_gradient(
            params, t, coupon, element
        )
    elif vj < bid:
        return (2 * (vj - bid) / (bid**2)) * bond_valuation_gradient(
            params, t, coupon, element
        )
    else:
        return 0.0

# Hessian Function


In [ ]:
@njit(cache=True)
def discount_factor_hessian(
    params: np.ndarray, t: np.ndarray, elements: List[int]
) -> np.ndarray:
    """Calculate hessian of discount factor"""
    f0, f1, f2, gamma = params
    sorted_elements = sorted(elements)
    if sorted_elements == [0, 0]:
        return t**2 * discount_factor(params, t)
    elif sorted_elements == [0, 1]:
        return -t * (-gamma + np.exp(-t / gamma) * gamma) * discount_factor(params, t)
    elif sorted_elements == [0, 2]:
        return (
            -t
            * (-gamma + np.exp(-t / gamma) * (t + gamma))
            * discount_factor(params, t)
        )
    elif sorted_elements == [0, 3]:
        return (
            -t
            * (
                -f1 * (1 - np.exp(-t / gamma) - t * np.exp(-t / gamma) / gamma)
                - f2
                * (
                    1
                    - np.exp(-t / gamma)
                    - t * np.exp(-t / gamma) * (t + gamma) / (gamma**2)
                )
            )
            * discount_factor(params, t)
        )
    elif sorted_elements == [1, 1]:
        return ((-gamma + np.exp(-t / gamma) * gamma) ** 2) * discount_factor(params, t)
    elif sorted_elements == [1, 2]:
        return (
            (-gamma + np.exp(-t / gamma) * gamma)
            * (-gamma + np.exp(-t / gamma) * (t + gamma))
            * discount_factor(params, t)
        )
    elif sorted_elements == [1, 3]:
        return (
            (-1 + np.exp(-t / gamma) + np.exp(-t / gamma) * t / gamma) * (f1 + 1)
            - f2
            * (
                1
                - np.exp(-t / gamma)
                - t * (t + gamma) * np.exp(-t / gamma) / (gamma**2)
            )
        ) * discount_factor(params, t)
    elif sorted_elements == [2, 2]:
        return ((-gamma + np.exp(-t / gamma) * (t + gamma)) ** 2) * discount_factor(
            params, t
        )
    elif sorted_elements == [2, 3]:
        return (
            (-1 + np.exp(-t / gamma) + np.exp(-t / gamma) * t / gamma)
            * (1 + f1 * (-gamma + np.exp(-t / gamma) * (t + gamma)))
            - (-gamma + np.exp(-t / gamma) * (t + gamma))
            * (
                1
                - np.exp(-t / gamma)
                - np.exp(-t / gamma) * t * (t + gamma) / (gamma**2)
            )
            * f2
        ) * discount_factor(params, t)
    else:
        return (
            np.exp(-t / gamma) * (t**2) * f1 / (gamma**3)
            - (
                -2 * np.exp(-t / gamma) * t / (gamma**2)
                - np.exp(-t / gamma) * (t**2) * (t + gamma) / (gamma**4)
                + 2 * np.exp(-t / gamma) * t
                + (t + gamma) / (gamma**3)
            )
            * f2
            + (
                f1 * (1 - np.exp(-t / gamma) - np.exp(-t / gamma) * t / gamma)
                + f2
                * (
                    1
                    - np.exp(-t / gamma)
                    - np.exp(-t / gamma) * t * (t + gamma) / (gamma**2)
                )
            )
            ** 2
        ) * discount_factor(params, t)


@njit(cache=True)
def bond_valuation_hessian(
    params: np.ndarray, t: np.ndarray, coupon: float, elements: List[int]
) -> float:
    """Calculate hessian of bond valuation"""
    cf = np.ones_like(t) * coupon / 2
    cf[-1] += 1
    return 100 * np.sum(cf * discount_factor_hessian(params, t, elements))


@njit(cache=True)
def loss_function_component_hessian(
    params: np.ndarray,
    t: np.ndarray,
    coupon: float,
    bid: float,
    ask: float,
    elements: List[int],
) -> float:
    """Calculate gradient of bond valuation"""
    vj = bond_valuation(params, t, coupon)
    if vj > ask:
        return (2 * (vj - ask) / (ask**2)) * bond_valuation_hessian(
            params, t, coupon, elements
        ) + (2 / (ask**2)) * bond_valuation_gradient(
            params, t, coupon, elements[0]
        ) * bond_valuation_gradient(
            params, t, coupon, elements[1]
        )
    elif vj < bid:
        return (2 * (vj - bid) / (bid**2)) * bond_valuation_hessian(
            params, t, coupon, elements
        ) + (2 / (bid**2)) * bond_valuation_gradient(
            params, t, coupon, elements[0]
        ) * bond_valuation_gradient(
            params, t, coupon, elements[1]
        )
    else:
        return 0.0